In [3]:
import re
import numpy as np
import sys
import os
import pandas as pd
import string
import matplotlib.pyplot as plt 
import re
from tqdm import tqdm

import googletrans
from googletrans import Translator

translator = Translator()
text2 = "Die Vorlesung über Suchmaschinen ist großartig!"
print(translator.detect(text2))
print(translator.translate(text2))

Detected(lang=de, confidence=None)
Translated(src=de, dest=en, text=The lecture on search engines is great!, pronunciation=None, extra_data="{'confiden...")


In [4]:
document= pd.read_csv("PPTranslatedAbstract_Title.csv")
query = pd.read_csv("PPTranslatedQuery.csv")

In [5]:
document.head()

,Unnamed: 0.2,Unnamed: 0,Preprocessed_Abstract,Preprocessed_Title,Unnamed: 0.1,qid:,qstr,candidates,DBRECORDID,TITLE,...,PUBLPLACE,INSTITUTION,TITLE_,ABSTRACT_,TITLE_lan,ABSTRACT_lan,title_en,abstract_en,TITLE_lan code(memo),ABSTRACT_lan code(memo)
0,0,0,"['the', 'paper', 'begins', 'with', 'a', 'brief...","['the', 'beauty', 'we', 'love', 'mbsr', 'teach...",0,943,mbsr,M30928765,M30928765,['The beauty we love: MBSR teacher education g...,...,NaN,NaN,The beauty we love MBSR teacher education goin...,The paper begins with a brief overview of the ...,en,en,The beauty we love MBSR teacher education goin...,The paper begins with a brief overview of the ...,NaN,NaN
1,1,1,"['this', 'paper', 'is', 'a', 'response', 'to',...","['building', 'an', 'ark', 'creating', 'a', 've...",1,943,mbsr,M30928764,M30928764,['Building an ark: creating a vessel for the e...,...,NaN,NaN,Building an ark creating a vessel for the educ...,This paper is a response to an invitation by t...,en,en,Building an ark creating a vessel for the educ...,This paper is a response to an invitation by t...,NaN,NaN
2,2,2,"['objectivethis', 'study', 'aims', 'to', 'surv...","['the', 'use', 'of', 'online', 'mbsr', 'audio'...",2,943,mbsr,M33155250,M33155250,['The use of online MBSR audio in medical staf...,...,NaN,NaN,The use of online MBSR audio in medical staff ...,ObjectiveThis study aims to survey medical sta...,en,en,The use of online MBSR audio in medical staff ...,ObjectiveThis study aims to survey medical sta...,NaN,NaN
3,3,3,"['genetic', 'variations', 'of', 'breast', 'can...","['translational', 'genomic', 'research', 'the'...",3,943,mbsr,M30137607,M30137607,['Translational genomic research: the role of ...,...,NaN,NaN,Translational genomic research the role of gen...,Genetic variations of breast cancer survivors ...,en,en,Translational genomic research the role of gen...,Genetic variations of breast cancer survivors ...,NaN,NaN
4,4,4,"['backgroundmindfulnessbased', 'programs', 'ha...","['the', 'effect', 'of', 'mindfulnessbased', 's...",4,943,mbsr,M32848377,M32848377,['The Effect of Mindfulness-Based Stress Reduc...,...,NaN,NaN,The Effect of MindfulnessBased Stress Reductio...,BackgroundMindfulnessbased programs have shown...,en,en,The Effect of MindfulnessBased Stress Reductio...,BackgroundMindfulnessbased programs have shown...,NaN,NaN


In [6]:
query.head()

,Unnamed: 0.1,Unnamed: 0,Preprocessed_Query,qid:,qstr,candidates,qstr_lan,qstr_en,Language(memo),Preprocessing
0,0,0,['mbsr'],943,mbsr,"['M30928765', 'M30928764', 'M33155250', 'M3013...",en,mbsr,NaN,['mbsr']
1,1,1,['gender'],925,gender,"['M32275910', 'NLM101312283', 'M10231258', 'M3...",en,gender,NaN,['gender']
2,2,2,['lonliness'],407,einsamkeit,"['NLM9707686', 'NLM9215358', 'M27924693', 'M33...",de,lonliness,NaN,['lonliness']
3,3,3,['midwife'],123,midwife,"['M18612453', 'M32361389', 'M33000708', 'M3353...",en,midwife,NaN,['midwife']
4,4,4,['bgm'],220,bgm,"['M33384475', 'M33392927', 'M33634364', 'M3217...",en,bgm,NaN,['bgm']


In [7]:
# checking if the num of candidates is 10000(100*100)
print(sum([len(query['candidates'][i].split('\', \'')) for i in range(100)]))

10000


In [8]:
def remove_special(x):
    x = x.replace("[", "").replace("]","").replace("\'","").replace(" ","")
    return x

query['candidates_'] = [remove_special(x).split(',') for x in query['candidates']]

merged_df = document.merge(query.explode("candidates_"), how='left', left_on=('candidates','qid:'), right_on=('candidates_','qid:'))

In [14]:
merged_df

,Unnamed: 0.2,Unnamed: 0_x,Preprocessed_Abstract,Preprocessed_Title,Unnamed: 0.1_x,qid:,qstr_x,candidates_x,DBRECORDID,TITLE,...,Unnamed: 0.1_y,Unnamed: 0_y,Preprocessed_Query,qstr_y,candidates_y,qstr_lan,qstr_en,Language(memo),Preprocessing,candidates_
0,0,0,"['the', 'paper', 'begins', 'with', 'a', 'brief...","['the', 'beauty', 'we', 'love', 'mbsr', 'teach...",0,943,mbsr,M30928765,M30928765,['The beauty we love: MBSR teacher education g...,...,0,0,['mbsr'],mbsr,"['M30928765', 'M30928764', 'M33155250', 'M3013...",en,mbsr,NaN,['mbsr'],M30928765
1,1,1,"['this', 'paper', 'is', 'a', 'response', 'to',...","['building', 'an', 'ark', 'creating', 'a', 've...",1,943,mbsr,M30928764,M30928764,['Building an ark: creating a vessel for the e...,...,0,0,['mbsr'],mbsr,"['M30928765', 'M30928764', 'M33155250', 'M3013...",en,mbsr,NaN,['mbsr'],M30928764
2,2,2,"['objectivethis', 'study', 'aims', 'to', 'surv...","['the', 'use', 'of', 'online', 'mbsr', 'audio'...",2,943,mbsr,M33155250,M33155250,['The use of online MBSR audio in medical staf...,...,0,0,['mbsr'],mbsr,"['M30928765', 'M30928764', 'M33155250', 'M3013...",en,mbsr,NaN,['mbsr'],M33155250
3,3,3,"['genetic', 'variations', 'of', 'breast', 'can...","['translational', 'genomic', 'research', 'the'...",3,943,mbsr,M30137607,M30137607,['Translational genomic research: the role of ...,...,0,0,['mbsr'],mbsr,"['M30928765', 'M30928764', 'M33155250', 'M3013...",en,mbsr,NaN,['mbsr'],M30137607
4,4,4,"['backgroundmindfulnessbased', 'programs', 'ha...","['the', 'effect', 'of', 'mindfulnessbased', 's...",4,943,mbsr,M32848377,M32848377,['The Effect of Mindfulness-Based Stress Reduc...,...,0,0,['mbsr'],mbsr,"['M30928765', 'M30928764', 'M33155250', 'M3013...",en,mbsr,NaN,['mbsr'],M32848377
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9995,9995,"['chronic', 'inflammation', 'increased', 'oxid...","['the', 'effect', 'of', 'antioxidant', 'antiin...",9995,1172,low AND carb,M33499118,M33499118,['The Effect of Antioxidant and Anti-Inflammat...,...,99,99,"['low', 'carb']",low AND carb,"['M31098613', 'M29446894', 'M30694317', 'M3060...",en,low AND carb,NaN,"['low', 'carb']",M33499118
9996,9996,9996,"['objectiveto', 'describe', 'characteristics',...","['participant', 'characteristics', 'selfrepted...",9996,1172,low AND carb,M32713393,M32713393,['Participant characteristics and self-reporte...,...,99,99,"['low', 'carb']",low AND carb,"['M31098613', 'M29446894', 'M30694317', 'M3060...",en,low AND carb,NaN,"['low', 'carb']",M32713393
9997,9997,9997,"['backgroundreplacing', 'carbohydrate', 'with'...","['effect', 'of', 'glycemic', 'index', 'carbohy...",9997,1172,low AND carb,M27391484,M27391484,['Effect of glycemic index and carbohydrate in...,...,99,99,"['low', 'carb']",low AND carb,"['M31098613', 'M29446894', 'M30694317', 'M3060...",en,low AND carb,NaN,"['low', 'carb']",M27391484
9998,9998,9998,"['objectivemacronutrient', 'regulation', 'of',...","['hmonal', 'metabolic', 'effects', 'of', 'carb...",9998,1172,low AND carb,M30614551,M30614551,['Hormonal and metabolic effects of carbohydra...,...,99,99,"['low', 'carb']",low AND carb,"['M31098613', 'M29446894', 'M30694317', 'M3060...",en,low AND carb,NaN,"['low', 'carb']",M30614551


In [10]:
new_title= []
for i in tqdm(range(merged_df.shape[0])):
    try:
        new_title.append(translator.translate(merged_df['TITLE_'][i],  src=merged_df['TITLE_lan'][i], dest=merged_df['qstr_lan'][i]).text)
    except:
        new_title.append(merged_df['TITLE_'][i])

100%|██████████| 10000/10000 [1:11:21<00:00,  2.34it/s] 


In [16]:
merged_df['new_title']=new_title
merged_df.to_csv("new_title.csv")
#new_title.to_csv("new_title.csv")

In [30]:
new_abstract= []
for i in tqdm(range(merged_df.shape[0])):
    try:
        new_abstract.append(translator.translate(merged_df['ABSTRACT_'][i],  src=merged_df['ABSTRACT_lan'][i], dest=merged_df['qstr_lan'][i]).text)
    except:
        new_abstract.append(merged_df['ABSTRACT_'][i])

100%|██████████| 10000/10000 [28:01<00:00,  5.95it/s] 


In [32]:
merged_df['new_abstract'] = new_abstract
merged_df
merged_df.to_csv("final_doc_query.csv")